In [15]:
import cv2
import os
import numpy as np
from collections import Counter

# Import training vectors

In [16]:
# 返回的是480张100*100的矩阵，和index
listdir = os.listdir('./face_train')
names = [d for d in listdir if not d.startswith('.')]
print(names)
images = []
target = []
for index,dir in enumerate(names):
    for i in range(1, 11):
        gray = cv2.imread('./face_train/%s/%d.jpg' % (dir, i))  # 三维图片
        gray_ = gray[:, :, 0]  # 二维数组
        gray_ = cv2.resize(gray_, dsize=(100, 100))
        gray_ = cv2.equalizeHist(gray_) # 图片均衡化处理，颜色鲜明      
        images.append(gray_)
        target.append(index)
images = np.asarray(images)
target = np.asarray(target)

['Yu Bin', 'Li Baoyan', 'Yu Ying', 'Yu Shaozhou', 'Yu Hong', 'Lv Hui', 'Wu Di', 'Dong Yini', 'Chen Jiayin', 'Wu Leru', 'Yu Di', 'Gao Yiqing', 'Liu Mingzhen', 'Li Xiufang', 'Chen Aijun', 'Jiang Letian', 'Yu Wei', 'Liu Jianhua', 'Zheng Ruojun', 'Dong Zhiyong']


In [17]:
# 转换成矩阵
#图像数据转换特征矩阵
image_data = []

for image in images:
    #转换成一维的数组
    data = image.flatten()
    image_data.append(data)

In [18]:
#转换为numpy数组
X = np.array(image_data)
print(type(X))
print(X.shape)
print(X)

<class 'numpy.ndarray'>
(200, 10000)
[[255 254 254 ...   8   9   9]
 [255 255 255 ...   4   7   9]
 [255 255 255 ...   4   6   8]
 ...
 [ 48  51  52 ...  14  19  23]
 [ 50  51  51 ...  18  26  41]
 [ 46  48  49 ...  18  30  48]]


# 将训练数据进行PCA

In [19]:
# 对数据进行中心化，即每一列减去该列的均值
X_train_centered = X - np.mean(X, axis=0)

In [20]:
# 主成分数量
k = 100
# 对训练矩阵进行SVD
U, S, Vt = np.linalg.svd(X_train_centered, full_matrices=False)
# 选择前100个主成分
X_train_pca = U[:, :k]
print(X_train_pca.shape)
# 构建变换矩阵P
P = np.dot(X.T, X_train_pca)
print(P)

(200, 100)
[[ 2.41686056e+02 -8.33475940e+01  2.05356429e+02 ...  2.60948812e+00
  -3.19586345e+00 -2.50127513e+00]
 [ 3.47248551e+02 -1.61956895e+02  2.36021146e+02 ... -1.38637901e+01
   1.22945201e+01 -1.77868372e+01]
 [ 4.62110815e+02 -2.24448074e+02  2.13469963e+02 ...  4.30605808e+00
  -8.52918663e-02 -1.64374631e+01]
 ...
 [ 8.27422850e+02 -3.64675595e+02  2.39867753e+02 ...  6.04117444e+00
   1.29298207e+01  4.12998862e+01]
 [ 7.69656850e+02 -4.14806874e+02  2.20282575e+02 ... -1.07580893e+01
   3.13039082e+01  6.15909583e+01]
 [ 7.33487268e+02 -4.06986750e+02  2.65872211e+02 ... -1.90051456e+01
   3.69763507e+01 -2.13712618e+01]]


In [21]:
cap = cv2.VideoCapture(0)
cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml'
face_detector = cv2.CascadeClassifier(cascade_path)
while True:
    flag,frame = cap.read()
    if not flag:
        break
   
    gray = cv2.cvtColor(frame,code = cv2.COLOR_BGR2GRAY)
    
    # 提取测试向量
    faces = face_detector.detectMultiScale(gray, minNeighbors=12)
    for x,y,w,h in faces:       
        
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face,dsize=(100,100))
        #face = cv2.equalizeHist(face)
        
        imgs = []
        imgs.append(face)
        imgs = np.asarray(imgs)
        
        #图像数据转换特征矩阵
        image_data = []

        data = imgs.flatten()
        image_data.append(data)
        
        #转换为numpy数组
        test_vector = np.array(image_data)
        #print(test_vector)

        # 归一化测试向量
        test_vector_centered = test_vector - np.mean(test_vector)
        #print(test_vector_centered)
        
        # 对测试向量PCA
        test_vector_pca = np.dot(test_vector_centered,P)
        
        #print(test_vector_pca)

        # 初始化一个列表来存储每个训练向量与当前测试向量的欧几里得范数
        distances = []

        for j in range(len(X_train_pca)):
            train_vector = X_train_pca[j]

            dist = np.sqrt(np.sum((train_vector-test_vector_pca) ** 2))
            distances.append(dist)

        # 找到最近的k个邻居
        nearest_indices = np.argsort(distances)[:3]
        print(nearest_indices)
        # 找到最频繁的索引
        counts = Counter(nearest_indices//10)
        most_common_index = counts.most_common(1)[0][0]
        
        print(most_common_index)

        # 人脸辨识，返回index和置信度
        y_ = most_common_index
        
        label = names[y_]
        
        print('这个人是：%s。'%(label))
        cv2.rectangle(frame,pt1 = (x,y),pt2 = (x+w,y+h),color=[0,0,255], thickness = 2)
        cv2.putText(frame,text = label, org = (x, y-10),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1.5,
                    color=[0,0,255],
                    thickness=2)
    cv2.imshow('face',frame)
    key = cv2.waitKey(1000//30)
    
    if key == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

[129 125 121]
12
这个人是：Liu Mingzhen。
[129 125 121]
12
这个人是：Liu Mingzhen。
[129 125 121]
12
这个人是：Liu Mingzhen。
[129 125 121]
12
这个人是：Liu Mingzhen。
[129  25  22]
2
这个人是：Yu Ying。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[125 129 128]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[129 125 127]
12
这个人是：Liu Mingzhen。
[125 129 127]
12
这个人是：Liu Mingzhen。
[125 129 127]
12
这个人是：Liu Mingzhen。
[125 129 127]
12
这个人是：Liu Mingzhen。
[125 129 127]
12
这个人是：Liu Mingzhen。
[129 127 125]
12
这个人是：Liu Mingzhen。
[127 125 129]
12
这个人是：Liu Mingzhen。
[ 39 127 125]
12
这个人是：Liu Mingzhen。
[ 39 127 125]
12
这个人是：Liu Mingzhen。
[127 125 129]
12
这个人是：Liu Mingzhen。
[125 129 127]
12
这个人是：Liu Mingzhen。
[127 125 129]
12
这个人是：Liu Mingzhen。
[127 125 129]
12
这个人是：Liu Mingzhen。
[125 129 127]
12
这个人是：Liu Mingzhen

KeyboardInterrupt: 